If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets as well as other dependencies. Uncomment the following cell and run it.

In [1]:
!pip install datasets evaluate transformers rouge-score nltk transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [30]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
import transformers

print(transformers.__version__)

4.31.0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="pytorch")

# Fine-tuning a model on a summarization task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a summarization task. We will use the [XSum dataset](https://arxiv.org/pdf/1808.08745.pdf) (for extreme summarization) which contains BBC articles accompanied with single-sentence summaries.

![Widget inference on a summarization task](https://github.com/huggingface/notebooks/blob/main/examples/images/summarization.png?raw=1)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `Trainer` API.

In [6]:
model_checkpoint = "facebook/bart-base"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`t5-small`](https://huggingface.co/t5-small) checkpoint.

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [7]:
!wget https://raw.githubusercontent.com/LasseRegin/medical-question-answer-data/master/ehealthforumQAs.json

--2023-07-24 06:51:38--  https://raw.githubusercontent.com/LasseRegin/medical-question-answer-data/master/ehealthforumQAs.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246142 (240K) [text/plain]
Saving to: ‘ehealthforumQAs.json’

ehealthforumQAs.jso 100%[===================>] 240.37K  --.-KB/s    in 0.003s  

2023-07-24 06:51:38 (78.6 MB/s) - ‘ehealthforumQAs.json’ saved [246142/246142]



In [8]:
!pip install -q nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.6 MB/s eta 0:00:00


In [9]:
import datasets
from datasets import load_dataset
from evaluate import load

train_ds = load_dataset("json", data_files=["ehealthforumQAs.json"], split="train[:80%]")
vals_ds = load_dataset("json", data_files=["ehealthforumQAs.json"], split="train[80%:]")
# raw_datasets = load_dataset("xsum")
raw_datasets = load_dataset("json", data_files={"train": "ehealthforumQAs.json", "validation": "ehealthforumQAs.json"})
metric = load("rouge")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-777ab9c218cd93cd/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-75657d766c7e278d/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# import nlp
# dataset_cc = nlp.concatenate_datasets([train_ds, vals_ds])

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [11]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'tags', 'url'],
        num_rows: 171
    })
    validation: Dataset({
        features: ['answer', 'question', 'tags', 'url'],
        num_rows: 171
    })
})

To access an actual element, you need to select a split first, then give an index:

In [12]:
raw_datasets["train"][0]

{'answer': 'stopping smoking is about will power and being steadfast. you can stop safely by having bupropion or nicotine patch cover initially in consult with a doctor. contact an addiction clinic near you. wishing you best of health thanks',
 'question': 'how do i stop smoking now',
 'tags': ['addiction', 'stop smoking'],
 'url': 'http://ehealthforum.com/health/stop-smoking-question-t462882.html'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [13]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [14]:
show_random_elements(raw_datasets["train"])

,answer,question,tags,url
0,since you have not had sea food chances of allergy to sea food are there if you take it not knowing the food allergy status. hence i suggest you to avoid sea food and take other sources of omega 3 supplements like walnuts flaxseeds hemp oil soyabean oil and pumpkin seeds. it is very difficult to precisely confirm a diagnosis without examination and investigations and the answer is based on the medical information provided. for exact diagnosis you are requested to consult your doctor. i sincerely hope that helps. take care.,i don't eat seafood. should i be taking omega 3 supplements/vitamins or should i get it from another source such as walnuts?,"[taking supplements, vitamins]",http://ehealthforum.com/health/omega-supplements-or-get-it-from-another-source-t348149.html
1,chia is grown commercially for its seed a food that is rich in omega-3 fatty acids î±-linolenic acid (ala). a ounce (28 g) of dried chia seeds contain 9% of the daily value for protein (4g) 13% fat (9g) (57% of which is ala) and 42% dietary fiber (11g) based on a daily intake of 2000 calories. the seeds also contain the essential minerals phosphorus manganese calcium potassium and sodium. taking this supplement will not affect you in a negative way inspite of you having coliac disease.,hello i have celiac disease and i was wondering if chia seeds interact negatively with my condition. also i heard that chia seeds have a blood pressure lowering effect - sometimes dangerously so. any advice would be welcomed!,[celiac disease],http://ehealthforum.com/health/will-chia-seeds-interact-negatively-with-my-condition-t298380.html
2,since the confirmatory tests done after the initial test are negative there is nothing for you to worry about. your obgyn is asking you to see a specialist just to make sure that anything is not missed. you should be just fine.,during a routine pap smear i requested hiv and std testing. all my std tests came back negative but the p24 antigen part of the ag/ab combo test came back reactive while the antibody test was negative. i then had a pcr test conducted on a new blood sample and the results were negative/nonreactive. i then had another ag/ab combo test performed where the p24 antigen and antibody test were both negative. 3 months later my doctor ordered another ab/ag combo test. the results were the same as the very first test p24 antigen negative/antibody negative. at this point i've been told by a pathologist that the positive tests are false and that my body must produce something that looks like the p24 antigen to the test. on the other hand my gyno wants me to go to an infections control specialist. i am so confused and i don't know what to do. i don't know how to definitively get an answer.,"[hiv and aids, hiv and aids]",http://ehealthforum.com/health/hiv-and-aids-question-t513730.html
3,the urge to urinate could suggest a presence of a lower urinary tract infection but in your case it is still too early to suppose any diagnosis because your symptoms have been lasting only for one night. if your symptoms continue or urination becomes painful you may want to consult your gp.,hello tonight i went to the bathroom normally to urinate while im going i feel a real warm feeling in the penis while im going. all of a sudden i am finished going and the pee feeling is still there and i don't have to go. sometimes i will go and a little will come out but the peeing feeling remains in the penis when i don't have to go and nothing comes out. i am only 23 so i would think im to young for prostate problems and the partner i have sex with is a trusting partner who says she has only been with me and which i am sure of and also the last sexual situtation was over a week from when this happened. this has just came out of nowhere and is kind of scaring me. i have no idea what is wrong. why is there a continious feeling that i have to urinate when i do not have to? please help. thank you,"[diagnosis, infection, prostate, sympto

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [15]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [16]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [18]:
tokenizer("Hello, this one sentence!")

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [19]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them using the `text_target` parameter. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [20]:
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [21]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [22]:
max_input_length = 4096
max_target_length = 2048

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["answer"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [23]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[0, 9178, 109, 939, 912, 7893, 122, 2], [0, 118, 56, 10, 17465, 337, 18541, 204, 107, 536, 8, 67, 33, 10, 3694, 403, 9, 253, 11174, 1069, 13310, 939, 524, 129, 974, 8, 127, 675, 2294, 2198, 155, 377, 536, 117, 55, 31571, 11, 227, 1085, 939, 524, 45, 5882, 117, 464, 11, 1230, 10095, 5626, 2408, 1085, 70, 20987, 6690, 3457, 33, 57, 2430, 99, 115, 28, 1593, 19, 162, 116, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[0, 620, 26307, 7893, 16, 59, 40, 476, 8, 145, 25781, 4, 47, 64, 912, 7385, 30, 519, 10306, 27128, 1499, 50, 27730, 9202, 1719, 3225, 11, 12777, 19, 10, 3299, 4, 1511, 41, 7530, 8474, 583, 47, 4, 17802, 47, 275, 9, 474, 2446, 2], [0, 42891, 42, 4428, 1341, 21942, 14, 528, 7, 117, 1219, 23, 42, 1046, 47, 33, 117, 4943, 13, 94, 130, 377, 4, 6690, 16, 8

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [24]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/171 [00:00<?, ? examples/s]

Map:   0%|          | 0/171 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [25]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [26]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/t5-finetuned-xsum"` or `"huggingface/t5-finetuned-xsum"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [28]:
import nltk
import numpy as np
import nltk
nltk.download('punkt')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [31]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/GuysTrans/bart-base-finetuned-xsum into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.40k/532M [00:00<?, ?B/s]

Download file runs/Jul24_06-41-32_c4cfcbb1328b/events.out.tfevents.1690180899.c4cfcbb1328b.278.1: 100%|#######…

Download file training_args.bin: 100%|##########| 4.06k/4.06k [00:00<?, ?B/s]

Download file runs/Jul24_06-46-50_c4cfcbb1328b/events.out.tfevents.1690181214.c4cfcbb1328b.278.2: 100%|#######…

Clean file runs/Jul24_06-41-32_c4cfcbb1328b/events.out.tfevents.1690180899.c4cfcbb1328b.278.1:  17%|#6        …

Clean file training_args.bin:  25%|##4       | 1.00k/4.06k [00:00<?, ?B/s]

Clean file runs/Jul24_06-46-50_c4cfcbb1328b/events.out.tfevents.1690181214.c4cfcbb1328b.278.2:  17%|#6        …

Clean file pytorch_model.bin:   0%|          | 1.00k/532M [00:00<?, ?B/s]

We can now finetune our model by just calling the `train` method:

In [32]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.690530,8.696200,1.061900,6.619500,7.933900,19.555600


TrainOutput(global_step=22, training_loss=4.643498160622337, metrics={'train_runtime': 23.7141, 'train_samples_per_second': 7.211, 'train_steps_per_second': 0.928, 'total_flos': 24163784386560.0, 'train_loss': 4.643498160622337, 'epoch': 1.0})

In [33]:
from tabulate import tabulate

In [34]:
model_before_finetuned = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [56]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["question"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=512)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [57]:
test_samples = vals_ds.select(range(8))


summaries_before_tuning = generate_summary(test_samples, model_before_finetuned)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

In [58]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Answer after", "Answer before"],
    )
)
print("\nTarget answers:\n")
print(
    tabulate(list(enumerate(test_samples["answer"])), headers=["Id", "Target answer"])
)
print("\nSource questions:\n")
print(tabulate(list(enumerate(test_samples["question"])), headers=["Id", "Question"]))

In [59]:
summaries_after_tuning

["hi i am from new delhi india. my dad was diagnosed with inoperable stage 4 metastatic pancreatic cancer on 12/03/2012 while he was in operation theater. he was admitted to the hospital on 12. 03. 2012. the doctors told him that he should undergo a wipple procedure to remove the tumor on the head of his pancreas or liver. the surgeon did not remove the tumour on his liver. he did not open him up for further treatment. the surgery was not successful. the tumor is still present in his liver and it is still growing. it is not clear if the tumor has spread to the liver or not. it may be that the tumor may have spread to other parts of the liver. it could be that there is a tumor in the liver that is not present in the brain. it can also be a tumor that may be present in other organs. there is also a possibility that there may be a tumour that is present in some parts of his body that may have been removed by the surgeon. this is a very difficult situation to deal with. please help me. ple

In [62]:
list(enumerate(test_samples["answer"]))

[(0,
  "hi gagan welcome to the ehealthforum and i am really glad to help you out. i can understand your concern for your uncle but the surgeon was absolutely right in abandoning the whipple's procedure on seeing the metastasis to the liver. once the tumor has metastasized to the liver it becomes a stage 4 pancreatic cancer. unfortunately majority of the patients don't live more than 6 months and only a few survive up to one year once the diagnosis is confirmed. the only option available presently is chemotherapy. i sincerely hope that helps. take care."),
 (1,
  'most gi illnesses don\'t cause pain in one\'s arm and so i would suggest that your doctors make sure that this pain you are experiencing is not related to any potential heart disease. beyond this suggestion there are many potential causes of upper abdominal pain including acid reflux (which almost always improves on acid-reducting medications) helicobacter pylroi infection (which requires both acid-reducing medications and an

In [60]:
# from transformers import pipeline, Conversation

# chatbot = pipeline("conversational", model=model, tokenizer=tokenizer)
# conversation = Conversation("hello am 22 years old and i have type 2diabet i wanted to sign for a new gym which use electric vibes to improve heart pulses and fasten the process of loosing weight i wanted to know if it is dangerous for me knowing that 20min of this sports equals 4 h of normal one thanks	")
# conversation = chatbot(conversation)
# conversation.generated_responses[-1]

You can now upload the result of the training to the Hub, just execute this instruction:

In [61]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```